In [24]:
!pip install nltk
import nltk
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords  
from nltk.corpus import words
from collections import Counter 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Gensim
!pip install -U gensim==3.8.3
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# installing mallet
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# !unzip mallet-2.0.8.zip


import pandas as pd
import numpy as np
import datetime as dt

# Plotting tools
import matplotlib
from matplotlib import pylab
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Center outputs
from IPython.core.display import HTML, Markdown
HTML("""
<style>
.output_png, .output {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

import warnings
warnings.filterwarnings('once')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [43]:
df=pd.read_csv("../data/data_news.csv")

In [44]:
df=df.drop(columns=['Unnamed: 0','titulo'])
df.head()

,categoria,href,fecha,title
0,Economía,/economia/david-stern-el-hombre-que-transformo...,2020-01-01,"David Stern, the man who transformed the NBA i..."
1,Internacional,/mundo/internacional/como-un-pais-con-vinos-de...,2020-01-01,How a country with unknown wines conquered New...
2,Empresas,/economia/empresas/hombre-mas-rico-de-asia-ret...,2020-01-01,Asia's richest man challenges Amazon with new ...
3,Empresas,/economia/empresas/italia-inicia-el-camino-par...,2020-01-01,Italy begins the path to suspend concessions t...
4,Política,/peru/politica/elecciones-2020-jne-confirma-do...,2020-01-01,Elections 2020: JNE confirms two candidacies a...


In [45]:
def convertToDatetime(x):
  dt_obj = dt.datetime.strptime(x, '%Y-%m-%d')
  return dt_obj
# converting all the rows's dates to datetime objects
df['fecha'] = df['fecha'].apply(convertToDatetime)
df['year'] = df['fecha'].apply((lambda x: x.year))

In [59]:
def remove_stopwords(texts):
    stop_words = stopwords.words('english')
    return [[word for word in simple_preprocess(str(doc)) if (word not in stop_words) & (len(word) > 2)] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# funciton to remove numbers
def removeNumbers(s):
  pattern = r'[0-9]'
  return re.sub(pattern, '', s)

# funciton to remove punctuation
def removePunctuation(s):
  pattern = r'[^\w\s]'
  return re.sub(pattern, '', s)

# function that lemmatizes and stems an array of words for later analysis
def lemmatizeStemArryOfWords(arr):
  result = [lemmatizer.lemmatize(word) for word in arr]
  result = [ps.stem(word) for word in result]
  return result

In [46]:
headlines = []
for fecha in df['fecha'].unique():
    headlines.append(' '.join(str(x) for x in df[df['fecha'] ==fecha ].iloc[0:16,3]))
    
    

In [47]:
data = pd.DataFrame(list(zip(df['fecha'].unique(), headlines)), columns=['Fecha', 'headlines'])

In [49]:
label = pd.read_excel("../data/MONTO DE INTERVENCION.xlsx")

In [35]:
label.head()

,Fecha,Monto de intervención,Tipo de Cambio,tasa,label,TCVENTA,Precio,Volumen
0,2020-12-28,7,3.612833,NaN,NaN,3.616167,20842.40,10630205
1,2020-12-29,52,3.617500,0.001291,1.0,3.619500,20862.78,10632724
2,2020-12-30,36,3.618167,0.000184,1.0,3.619833,20967.19,5528837
3,2020-12-31,0,3.620250,0.000576,1.0,3.623000,20822.15,4391459
4,2021-01-04,142,3.624000,0.001035,1.0,3.626667,21013.24,23568144


In [51]:
data1 = label.merge(data,on='Fecha', how='left')

In [55]:
data1 = data1.iloc[1:,:]

In [58]:
data1.head()

,Fecha,Monto de intervención,Tipo de Cambio,tasa,label,TCVENTA,Precio,Volumen,headlines
1,2020-12-29,52,3.617500,0.001291,1.0,3.619500,20862.78,10632724,Approving interest rate control law is a setba...
2,2020-12-30,36,3.618167,0.000184,1.0,3.619833,20967.19,5528837,FTA with the United Kingdom favors the economi...
3,2020-12-31,0,3.620250,0.000576,1.0,3.623000,20822.15,4391459,They admit to process a cross against the cand...
4,2021-01-04,142,3.624000,0.001035,1.0,3.626667,21013.24,23568144,Elections 2021: Two objections presented again...
5,2021-01-05,35,3.627500,0.000965,1.0,3.633667,21009.13,13526944,Part of the list of candidates for the Lima Co...


In [65]:
data1 = data1.loc[:,('Fecha','label','headlines')]

In [70]:
data1.iloc[164,:]

Fecha                                      2021-08-24 00:00:00
label                                                      0.0
headlines    Vladimiro Montesino will be transferred to the...
Name: 165, dtype: object

In [73]:
train  = data[data['Fecha'] < '2021-08-24']
test = data[data['Fecha'] >= '2021-08-24']

In [76]:
text = train.loc[:,'headlines']

In [78]:
text.replace("[^a-zA-Z]"," ", regex = True, inplace = True)

C:\Users\PC\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [79]:
text

0      David Stern  the man who transformed the NBA i...
1      Keiko Fujimori  judge rejects psychiatric expe...
2      JNJ  Prosecutor of the Nation voted against th...
3      Elections       citizens will be able to ask q...
4      Gra a y Montero knew of US   million delivered...
                             ...                        
596    JNJ asks the Executive Branch for a report fro...
597    Colombia will give booster doses against COVID...
598    Learn about these digital solutions that will ...
599    SNMPE      of mining profits stay in Peru Elme...
600    Roberto S nchez   An attempt is being made to ...
Name: headlines, Length: 601, dtype: object